In [15]:
from IPython.display import display, Markdown
with open('DiffusionProblemOnSquare.md', 'r') as file1:
    DiffusionProblemOnSquare = file1.read()
with open('DescriptionFV5DiffusionProblem.md', 'r') as file2:
    DescriptionFV5DiffusionProblem = file2.read()
with open('CodeFV5DiffusionProblem.md', 'r') as file3:
    CodeFV5DiffusionProblem = file3.read()

# FV5 scheme for a Diffusion equation

In [16]:
display(Markdown(DiffusionProblemOnSquare))

## The Diffusion problem on the square

We consider the following diffusion problem with Dirichlet boundary conditions

$$
\left\{\begin{array}{c}
-\left(\partial_{xx} u + K\partial_{yy} u\right) = f \textrm{ on } \Omega\\
u=0 \textrm{ on } \partial\Omega
\end{array}\right.
$$

on the square domain $\Omega= [0,1]\times [0,1]$ with  

$$f=(1+K)\pi^2 sin(\pi x) sin(\pi y).$$  
The unique solution of the problem is  

$$
u=sin(\pi x) sin(\pi y).
$$

The Diffusion equation can be written in a matrix form
$$
-\nabla\cdot(D\vec\nabla u)=f
$$
and the associated diffusion matrix is
$$
D=\left(\begin{array}{cc}
    1 & 0\\
    0 & K
  \end{array}\right)
$$

We are interested in case where $K\gg 1$. In the following numerical results we take the value $K=10^4$.


In [17]:
display(Markdown(DescriptionFV5DiffusionProblem))

## The FV5 scheme for the Diffusion equation

The domain $\Omega$ is decomposed into cells $C_i$.

$|C_i|$ is the measure of the cell $C_i$.

$f_{ij}$ is the interface between two cells $C_i$ and $C_j$. 

$\vec n_{ij}$ is the normal vector to the interface between two cells $C_i$ and $C_j$. 

$s_{ij}$ is the measure of the interface $f_{ij}$.

$d_{ij}$ is the distance between the centers of mass of the two cells $C_i$ and $C_j$.

The discrete Diffusion problem is
$$
-\frac{1}{|C_i|} \sum s_{ij}F_{ij}=f_i,
$$
where
$u_i$ is the approximation of $u$ in the cell $C_i$,

$f_i$ is the approximation of $f$ in the cell $C_i$,

$F_{ij}$ is a numerical approximation of the outward normal diffusion flux from cell $i$ to cell $j$.

In the case of the scheme FV5, the flux formula are
$$
F_{ij}=\frac{u_j-u_i}{d_{ij}} {}^t\vec n_{ij}D\vec n_{ij},
$$
for two cells $i$ and $j$ inside the domain,

and
$$
F_{boundary}=\frac{u(x_f)-u_i}{d_{if}} {}^t\vec n_{if}D\vec n_{if},
$$
for a boundary face with center $x_f$, inner cell $i$, outer normal vector $\vec n_{ij}$ and distance between face and cell centers $d_{if}$


In [18]:
display(Markdown(CodeFV5PoissonProblem))

## The script

```python
#Discrétisation du second membre et extraction du nb max de voisins d'une cellule
#================================================================================
my_RHSfield = cdmath.Field("RHS_field", cdmath.CELLS, my_mesh, 1)
maxNbNeighbours=0#This is to determine the number of non zero coefficients in the sparse finite element rigidity matrix

for i in range(nbCells): 
	Ci = my_mesh.getCell(i)
	x = Ci.x()
	y = Ci.y()

	my_RHSfield[i]=2*pi*pi*sin(pi*x)*sin(pi*y)#mettre la fonction definie au second membre de l edp
	# compute maximum number of neighbours
	maxNbNeighbours= max(1+Ci.getNumberOfFaces(),maxNbNeighbours)

# Construction de la matrice et du vecteur second membre du système linéaire
#===========================================================================
Rigidite=cdmath.SparseMatrixPetsc(nbCells,nbCells,maxNbNeighbours)# warning : third argument is max number of non zero coefficients per line of the matrix
RHS=cdmath.Vector(nbCells)
normal=cdmath.Vector(dim)
#Parcours des cellules du domaine
for i in range(nbCells):
	RHS[i]=my_RHSfield[i] #la valeur moyenne du second membre f dans la cellule i
	Ci=my_mesh.getCell(i)
	for j in range(Ci.getNumberOfFaces()):# parcours des faces voisinnes
		Fj=my_mesh.getFace(Ci.getFaceId(j))
            		for idim in range(dim) :
                		normal[idim] = Ci.getNormalVector(j, idim);#normale sortante
		if not Fj.isBorder():
			k=Fj.getCellId(0)
			if k==i :
				k=Fj.getCellId(1)
			Ck=my_mesh.getCell(k)
			distance=Ci.getBarryCenter().distance(Ck.getBarryCenter())
			coeff=Fj.getMeasure()/Ci.getMeasure()/distance*(normal[0]*normal[0] + K*normal[1]*normal[1])
			Rigidite.setValue(i,k,-coeff) # terme extradiagonal
		else:
			coeff=Fj.getMeasure()/Ci.getMeasure()/Ci.getBarryCenter().distance(Fj.getBarryCenter())*(normal[0]*normal[0] + K*normal[1]*normal[1])
			#For the particular case where the mesh boundary does not coincide with the domain boundary
			x=Fj.getBarryCenter().x()
			y=Fj.getBarryCenter().y()
			RHS[i]+=coeff*sin(pi*x)*sin(pi*y)#mettre ici la condition limite du problème de Dirichlet
		Rigidite.addValue(i,i,coeff) # terme diagonal


# Résolution du système linéaire
#=================================
LS=cdmath.LinearSolver(Rigidite,RHS,500,1.E-6,"GMRES","ILU")
SolSyst=LS.solve()

# Automatic postprocessing :  save 2D picture and plot diagonal data
#===========================
PV_routines.Save_PV_data_to_picture_file("my_ResultField_0.vtu',"ResultField",'CELLS',"my_ResultField")
diag_data=VTK_routines.Extract_field_data_over_line_to_numpyArray(my_ResultField,[0,1,0],[1,0,0], resolution)
plt.plot(curv_abs, diag_data, label= str(nbCells)+ ' cells mesh')
plt.savefig("FV5_on_square_PlotOverDiagonalLine.png")

```


## Regular grid

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_squares/squareWithSquares_1.png) | ![](2DDiffusionVF_squares/squareWithSquares_2.png)  | ![](2DDiffusionVF_squares/squareWithSquares_3.png) 


result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_squares/FiniteVolumes2DDiffusion_SQUARE_Regular_squares16.png) | ![](2DDiffusionVF_squares/FiniteVolumes2DDiffusion_SQUARE_Regular_squares64.png)  | ![](2DDiffusionVF_squares/FiniteVolumes2DDiffusion_SQUARE_Regular_squares256.png) 


![](2DDiffusionVF_squares/squareWithSquares_2DDiffusionFV_ConvergenceCurve.png)

## Deformed quadrangles

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_deformedQuadrangles/squareWithDeformedQuadrangles_1.png) | ![](2DDiffusionVF_deformedQuadrangles/squareWithDeformedQuadrangles_2.png)  | ![](2DDiffusionVF_deformedQuadrangles/squareWithDeformedQuadrangles_3.png) 

result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_deformedQuadrangles/FiniteVolumes2DDiffusion_SQUARE_Deformed_quadrangles16.png) | ![](2DDiffusionVF_deformedQuadrangles/FiniteVolumes2DDiffusion_SQUARE_Deformed_quadrangles64.png)  | ![](2DDiffusionVF_deformedQuadrangles/FiniteVolumes2DDiffusion_SQUARE_Deformed_quadrangles256.png) 

![](2DDiffusionVF_deformedQuadrangles/squareWithDeformedQuadrangles_2DDiffusionVF_ConvergenceCurve.png)

## Delaunay triangular meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_Delaunay_triangles/squareWithTriangles_1.png) | ![](2DDiffusionVF_Delaunay_triangles/squareWithTriangles_2.png)  | ![](2DDiffusionVF_Delaunay_triangles/squareWithTriangles_3.png) 

result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_Delaunay_triangles/FiniteVolumes2DDiffusion_SQUARE_Unstructured_Delaunay_triangles40.png) | ![](2DDiffusionVF_Delaunay_triangles/FiniteVolumes2DDiffusion_SQUARE_Unstructured_Delaunay_triangles224.png)  | ![](2DDiffusionVF_Delaunay_triangles/FiniteVolumes2DDiffusion_SQUARE_Unstructured_Delaunay_triangles934.png) 

![](2DDiffusionVF_Delaunay_triangles/squareWithDelaunayTriangles_2DDiffusionVF_ConvergenceCurve.png)

## Cross triangle meshes (from a $(n,2n)$ rectangular grid)

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_cross_triangles/squareWithCrossTriangles_0.png) | ![](2DDiffusionVF_cross_triangles/squareWithCrossTriangles_1.png)  | ![](2DDiffusionVF_cross_triangles/squareWithCrossTriangles_2.png) 


result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_cross_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_cross_triangles72.png) | ![](2DDiffusionVF_cross_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_cross_triangles288.png)  | ![](2DDiffusionVF_cross_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_cross_triangles1800.png) 


![](2DDiffusionVF_cross_triangles/squareWithCrossTriangles_2DDiffusionFV_ConvergenceCurve.png)

## Hexagonal meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_hexagons/squareWithHexagons_1.png) | ![](2DDiffusionVF_hexagons/squareWithHexagons_2.png)  | ![](2DDiffusionVF_hexagons/squareWithHexagons_3.png) 

result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_hexagons/FiniteVolumes2DDiffusion_SQUARE_Regular_hexagons25.png) | ![](2DDiffusionVF_hexagons/FiniteVolumes2DDiffusion_SQUARE_Regular_hexagons255.png)  | ![](2DDiffusionVF_hexagons/FiniteVolumes2DDiffusion_SQUARE_Regular_hexagons1020.png) 

![](2DDiffusionVF_hexagons/squareWithHexagons_2DDiffusionVF_ConvergenceCurve.png)

## Locally refined meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_loc_ref/squareWithLocRefSquares_1.png) | ![](2DDiffusionVF_loc_ref/squareWithLocRefSquares_2.png)  | ![](2DDiffusionVF_loc_ref/squareWithLocRefSquares_3.png) 

result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_loc_ref/FiniteVolumes2DDiffusion_SQUARE_Non_conforming_locally_refined40.png) | ![](2DDiffusionVF_loc_ref/FiniteVolumes2DDiffusion_SQUARE_Non_conforming_locally_refined160.png)  | ![](2DDiffusionVF_loc_ref/FiniteVolumes2DDiffusion_SQUARE_Non_conforming_locally_refined640.png) 


![](2DDiffusionVF_loc_ref/squareWithLocRefSquares_2DDiffusionVF_ConvergenceCurve.png)

## Checkerboard meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_checkerboard/checkerboard_4x4.png) | ![](2DDiffusionVF_checkerboard/checkerboard_8x8.png)  | ![](2DDiffusionVF_checkerboard/checkerboard_16x16.png) 


result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_checkerboard/FiniteVolumes2DDiffusion_SQUARE_Non_conforming_checkerboard40.png) | ![](2DDiffusionVF_checkerboard/FiniteVolumes2DDiffusion_SQUARE_Non_conforming_checkerboard160.png)  | ![](2DDiffusionVF_checkerboard/FiniteVolumes2DDiffusion_SQUARE_Non_conforming_checkerboard640.png) 


![](2DDiffusionVF_checkerboard/squareWithCheckerboard_2DDiffusionVF_ConvergenceCurve.png)

## Brick wall meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DDiffusionVF_brickwall/squareWithBrickWall_1.png) | ![](2DDiffusionVF_brickwall/squareWithBrickWall_2.png)  | ![](2DDiffusionVF_brickwall/squareWithBrickWall_3.png) 


result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_brickwall/FiniteVolumes2DDiffusion_SQUARE_Regular_brickwall25.png) | ![](2DDiffusionVF_brickwall/FiniteVolumes2DDiffusion_SQUARE_Regular_brickwall225.png)  | ![](2DDiffusionVF_brickwall/FiniteVolumes2DDiffusion_SQUARE_Regular_brickwall961.png) 


![](2DDiffusionVF_brickwall/squareWithBrickWall_2DDiffusionVF_ConvergenceCurve.png)

## Long rectangle meshes ( $(n,n^2)$ rectangular grid)

mesh 1 | mesh 2 
     - | -    -
![](2DDiffusionVF_long_rectangles/squareWithLongRectangles_00.png) | ![](2DDiffusionVF_long_rectangles/squareWithLongRectangles_0.png) | ![](2DDiffusionVF_long_rectangles/squareWithLongRectangles_1.png) 

result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_long_rectangles/FiniteVolumes2DDiffusion_SQUARE_Regular_long_rectangles125.png) | ![](2DDiffusionVF_long_rectangles/FiniteVolumes2DDiffusion_SQUARE_Regular_long_rectangles1331.png)  | ![](2DDiffusionVF_long_rectangles/FiniteVolumes2DDiffusion_SQUARE_Regular_long_rectangles9261.png) 


![](2DDiffusionVF_long_rectangles/squareWithLongRectangles_2DDiffusionFV_ConvergenceCurve.png)

## Skinny right triangle meshes (from a $(n,n^2)$ rectangular grid)

mesh 1 | mesh 2 | mesh 3 
     - | -    - | -    - 
![](2DDiffusionVF_skinny_triangles/squareWithSkinnyTriangles_00.png) | ![](2DDiffusionVF_skinny_triangles/squareWithSkinnyTriangles_0.png) | ![](2DDiffusionVF_skinny_triangles/squareWithSkinnyTriangles_1.png) 

result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_skinny_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_skinny_triangles250.png) | ![](2DDiffusionVF_skinny_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_skinny_triangles1458.png)  | ![](2DDiffusionVF_skinny_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_skinny_triangles18522.png) 


![](2DDiffusionVF_skinny_triangles/squareWithSkinnyTriangles_2DDiffusionFV_ConvergenceCurve.png)

## Flat cross triangle meshes (from a $(n,n^2)$ rectangular grid)

mesh 1 | mesh 2 | mesh 3
     - | -    - | -    - 
![](2DDiffusionVF_flat_cross_triangles/squareWithFlatCrossTriangles_00.png) | ![](2DDiffusionVF_flat_cross_triangles/squareWithFlatCrossTriangles_0.png) | ![](2DDiffusionVF_flat_cross_triangles/squareWithFlatCrossTriangles_1.png) 

result 1 | result 2 | result 3
       - | -      - | -
![](2DDiffusionVF_flat_cross_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_flat_cross_triangles500.png) | ![](2DDiffusionVF_flat_cross_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_flat_cross_triangles5324.png)  | ![](2DDiffusionVF_flat_cross_triangles/FiniteVolumes2DDiffusion_SQUARE_Regular_flat_cross_triangles37044.png) 


![](2DDiffusionVF_flat_cross_triangles/squareWithFlatCrossTriangles_2DDiffusionFV_ConvergenceCurve.png)